In [1]:
import json
import pandas as pd
import numpy as np
import sys
import os

from helper_function.write_db import write_dataframe_to_sqlite
import helper_function.misc as helper
from helper_function.metadata import setting_category

In [2]:
# Function to try conversion to string, or return -1
def try_convert_to_float(value):
    if pd.isna(value):
        # Check for both None and NaN values
        return value
    try:
        # Attempt to convert to float
        return float(value)
    except ValueError:
        # If conversion fails, return -1
        return -1

    
def contains_alphabet(s):
    # Check if s is np.nan
    if isinstance(s, float) and np.isnan(s):
        return False

    s = str(s)
    return any(char.isalpha() for char in s)

In [4]:
file = "data/hospitcal_pricing/NYU Langone Orthopedic Hospital standard charges.csv"
data = pd.read_csv(file, skiprows=[0,1])

/var/folders/gq/f0m_7zn13nx9vghg_5vt_c700000gp/T/ipykernel_18478/741912902.py:2: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,

In [6]:
data.head()

,Identifier Code,Billing Code,Identifier Description,NYU Langone Gross Charges,NYU Langone Discounted Cash Price,AETNA HMO,AETNA INDEMNITY,BCBS GHI COMPOSITE,BCBS HMO,BCBS INDEMNITY,...,HEALTHFIRST ESSENTIAL PLAN 1&2,HEALTHFIRST GOLD LEAF EXCHANGE,HEALTHFIRST BRONZE LEAF EXCHANGE,HEALTHFIRST GREEN LEAF EXCHANGE,HEALTHFIRST PLATINUM LEAF EXCHANGE,HEALTHFIRST SILVER LEAF EXCHANGE,"HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO",HEALTHNET PPO,De-identified Minimum,De-identified Maximum
0,2023696,19120,HC EXC CYST/ABERRANT BREAST TISSUE OPEN 1/> LE...,7750.52,1472.60,15317.57,7750.52,11810.00,11810.00,11810.00,...,1792.32,1792.32,1792.32,1792.32,1792.32,1792.32,1792.32,6277.92,776.73,22768.0
1,2023358,43235,HC UGI DIAGNOSTIC,3344.09,635.38,6116.92,3344.09,6792.00,6792.00,6792.00,...,975.42,975.42,975.42,975.42,975.42,975.42,975.42,2708.71,248.40,8498.0
2,2023360,43239,HC UGI W BX. SGL/MULTIPLE,1464.39,278.23,7831.48,1464.39,6792.00,6792.00,6792.00,...,975.42,975.42,975.42,975.42,975.42,975.42,975.42,1186.16,276.43,8195.0
3,2023385,45378,HC DIAGNOSTIC COLONOSCOPY,1851.10,351.71,7831.48,1851.1,6792.00,6792.00,6792.00,...,941.57,941.57,941.57,941.57,941.57,941.57,941.57,1499.39,404.00,8195.0
4,2023387,45380,HC COLONOSCOPY AND BIOPSY,2206.10,419.16,7831.48,2206.1,6792.00,6792.00,6792.00,...,941.57,941.57,941.57,941.57,941.57,941.57,941.57,1786.94,481.11,8195.0


In [7]:
hospital_id = 330389
# melt
core_col = ['Identifier Code', 'Billing Code', 'Identifier Description']
melt_col = list(set(data.columns) - set(core_col))

data = data.melt(id_vars=core_col, value_vars=melt_col, var_name="payer_name", value_name="standard_charge")

# standard charge column cleaning 
data.standard_charge = data.standard_charge.str.replace(',', '')

data["additional_generic_notes"] = data.apply(lambda row: f"standard_charge: {row['standard_charge']}" if contains_alphabet(row["standard_charge"]) else None, axis =1)
data.standard_charge = data.standard_charge.apply(try_convert_to_float)

data.standard_charge = data.standard_charge.astype(float)

# replace payer_name to min max
data['payer_name'].mask(data['payer_name'] == 'De-identified Maximum', "Max_Negotiated_Rate", inplace=True)
data['payer_name'].mask(data['payer_name'] == 'De-identified Minimum', "Min_Negotiated_Rate", inplace=True)
data['payer_name'].mask(data['payer_name'] == 'NYU Langone Gross Charges', "Gross_Charge", inplace=True)
data['payer_name'].mask(data['payer_name'] == 'NYU Langone Discounted Cash Price', "Cash_Charge", inplace=True)

# Create rate category
data["rate_category"] = data.apply(helper.determine_rate_category, axis=1)

# Rename columns
data = data.rename(columns={"Billing Code":"code", "Identifier Code": "local_code", "Identifier Description": "description"})


In [9]:
data["additional_generic_notes"].value_counts()

additional_generic_notes
standard_charge: NSP                 7416320
standard_charge: POC                   43378
standard_charge: 73% of charges        35223
standard_charge: Cost                  21862
standard_charge: 5208 PD               16514
                                      ...   
standard_charge: $59.00 per visit          3
standard_charge: 12294 PD                  2
standard_charge: 9316 PD                   2
standard_charge: 3990 PD                   1
standard_charge: 3991 PD                   1
Name: count, Length: 74, dtype: int64

In [ ]:
data["hospital_id"] = hospital_id
write_dataframe_to_sqlite(data, 'rate', 'data/hospital_pricing.db')